In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from uncertainties import ufloat
import warnings


# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


In [4]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Kim_geldsetzer densities

In [5]:
# Collect relevant data from each snowpit
layer_info = []

for pit in all_pits:

    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer.depth_top[0] if layer.depth_top else None,  
            'thickness': layer.thickness[0] if layer.thickness else None,
        }

        # Add kim_geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer')
            try: 
                # Calculate density using kim_geldsetzer method - only if we have valid inputs
                if layer.hardness and layer_dict['kim_geldsetzer_grain_form']:
                    density_ufloat = density.calculate_density( 
                        method='kim_geldsetzer',
                        hand_hardness=layer.hardness,
                        grain_form=layer_dict['kim_geldsetzer_grain_form']
                    )
                    layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                    layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
                else:
                    layer_dict['density_kim_geldsetzer'] = None
                    layer_dict['density_kim_geldsetzer_uncertainty'] = None
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['kim_geldsetzer_grain_form'] = None
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        layer_info.append(layer_dict)

# Create dataframes
layer_df = pd.DataFrame(layer_info)


In [6]:
# Implement the elastic modulus calculations

for idx, row in layer_df.iterrows():
    if row['kim_geldsetzer_grain_form'] is not None:
        try:
            # Calculate elastic modulus using Bergfeld method
            E_mod_bergfeld = elastic_modulus.calculate_elastic_modulus(
                method='bergfeld',
                density=ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
            )
            layer_df.at[idx, 'e_mod_bergfeld'] = E_mod_bergfeld.nominal_value
            E_mod_kochle = elastic_modulus.calculate_elastic_modulus(
                method='kochle',
                density=ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty']),
                grain_form=row['kim_geldsetzer_grain_form']
            )
            layer_df.at[idx, 'e_mod_kochle'] = E_mod_kochle.nominal_value
            E_mod_wautier = elastic_modulus.calculate_elastic_modulus(
                method='wautier',
                density=ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty']),
                grain_form=row['kim_geldsetzer_grain_form']
            )
            layer_df.at[idx, 'e_mod_wautier'] = E_mod_wautier.nominal_value
        except Exception:
            layer_df.at[idx, 'e_mod_bergfeld'] = np.nan
            layer_df.at[idx, 'e_mod_kochle'] = np.nan
            layer_df.at[idx, 'e_mod_wautier'] = np.nan
